# Iterating on LLM Apps with TruLens

1. Start with basic RAG.
2. Show failures of RAG Triad.
3. Address failures with context filtering, advanced RAG (e.g., sentence windows, auto-retrieval)
4. Improve RAG with agents
5. Showcase experiment tracking to choose best app configuration. 
6. Weave in different types of evals into narrative
7. Weave in user/customer stories into narrative


## Notes:

Use Insurance policy as data.

In [1]:
# Set your API keys. If you already have them in your var env., you can skip these steps.
import os
import openai

os.environ["OPENAI_API_KEY"] = "..."
openai.api_key = os.environ["OPENAI_API_KEY"]

## Start with basic RAG.

In [2]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./Insurance_Handbook_20103.pdf"]
).load_data()

In [3]:
from llama_index import Document

from llama_index import ServiceContext, VectorStoreIndex, StorageContext

from llama_index.llms import OpenAI

# initialize llm
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.5)

# knowledge store
document = Document(text="\n\n".join([doc.text for doc in documents]))

from llama_index import VectorStoreIndex

# service context for index
service_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model="local:BAAI/bge-small-en-v1.5")

# create index
index = VectorStoreIndex.from_documents([document], service_context=service_context)

from llama_index import Prompt

system_prompt = Prompt("SYSTEM PROMPT: You are an insurance assistant who is charged with answering questions completely.\n"
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this system prompt and context, please answer the question: {query_str}\n")

# basic rag query engine
rag_basic = index.as_query_engine(text_qa_template = system_prompt)

## Load test set

In [4]:
# Load some questions for evaluation
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

## Set up Evaluation

In [8]:
import numpy as np
from trulens_eval import Tru, Feedback, TruLlama, OpenAI as fOpenAI

tru = Tru()

# start fresh
tru.reset_database()

from trulens_eval.feedback import Groundedness

openai = fOpenAI()

qa_relevance = (
    Feedback(openai.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input_output()
)

qs_relevance = (
    Feedback(openai.relevance_with_cot_reasons, name = "Context Relevance")
    .on_input()
    .on(TruLlama.select_source_nodes().node.text)
    .aggregate(np.mean)
)

from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=openai)

groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name="Groundedness")
        .on(TruLlama.select_source_nodes().node.text)
        .on_output()
        .aggregate(grounded.grounded_statements_aggregator)
)

feedbacks = [qa_relevance, qs_relevance, groundedness]

from trulens_eval import FeedbackMode

tru_recorder_rag_basic = TruLlama(
        rag_basic,
        app_id='Basic RAG',
        feedbacks=feedbacks
    )

Feedback function `groundedness_measure` was renamed to `groundedness_measure_with_cot_reasons`. The new functionality of `groundedness_measure` function will no longer emit reasons as a lower cost option. It may have reduced accuracy due to not using Chain of Thought reasoning in the scoring.


Deleted 8 rows.
✅ In Answer Relevance, input prompt will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to *.__record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to *.__record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to *.__record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to *.__record__.main_output or `Select.RecordOutput` .


In [6]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.4.23:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [9]:
# Run evaluation on 10 sample questions
with tru_recorder_rag_basic as recording:
    for question in eval_questions:
        response = rag_basic.query(question)

Groundedness scores: {'statement_0': 1.0}
Groundedness scores: {'statement_0': 0.0}
Groundedness scores: {'statement_0': 0.8}
Groundedness scores: {'statement_0': 0.8}
Groundedness scores: {'statement_0': 0.0}
Groundedness scores: {'statement_0': 1.0}
Groundedness scores: {'statement_0': 0.0}
Groundedness scores: {'statement_0': 1.0, 'statement_1': 1.0}
Groundedness scores: {'statement_0': 1.0, 'statement_1': 1.0, 'statement_2': 1.0, 'statement_3': 1.0, 'statement_4': 1.0}
Groundedness scores: {'statement_0': 0.0, 'statement_1': 0.0, 'statement_2': 0.0, 'statement_3': 0.0, 'statement_4': 0.0}
Groundedness scores: {'statement_0': 1.0, 'statement_1': 1.0, 'statement_2': 1.0}
Groundedness scores: {'statement_0': 1.0}
Groundedness scores: {'statement_0': 1.0}
Groundedness scores: {'statement_0': 0.7, 'statement_1': 1.0, 'statement_2': 1.0, 'statement_3': 1.0, 'statement_4': 1.0, 'statement_5': 0.0}
Groundedness scores: {'statement_0': 1.0, 'statement_1': 1.0, 'statement_2': 0.0}
Groundedne

Groundedness scores: {'statement_0': 0.0}
Groundedness scores: {'statement_0': 0.0}
Groundedness scores: {'statement_0': 0.0}
Groundedness scores: {'statement_0': 1.0, 'statement_1': 1.0, 'statement_2': 1.0, 'statement_3': 1.0, 'statement_4': 1.0}


In [9]:
#tru.stop_evaluator()

Our simple RAG often struggles with retrieving not enough information from the insurance manual to properly answer the question. The information needed may be just outside the chunk that is identified and retrieved by our app. Let's try sentence window retrieval to retrieve a wider chunk.

In [10]:
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage
import os

def build_sentence_window_index(
    document, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="sentence_index"
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=3,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            [document], service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index

sentence_index = build_sentence_window_index(
    document, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="sentence_index"
)

def get_sentence_window_query_engine(
    sentence_index,
    similarity_top_k=6,
    rerank_top_n=2,
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

tru_recorder_rag_sentencewindow = TruLlama(
        sentence_window_engine,
        app_id='RAG - Sentence Window',
        feedbacks=feedbacks
    )

In [11]:
# Run evaluation on 10 sample questions
with tru_recorder_rag_sentencewindow as recording:
    for question in eval_questions:
        response = sentence_window_engine.query(question)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1a64d8b50 is calling an instrumented method <function BaseQueryEngine.query at 0x138000a40>. The path of this call may be incorrect.
Guessing path of new object is *.app based on other object (0x1a3e1bd90) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1a64d8b50 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x1a3e693a0>. The path of this call may be incorrect.
Guessing path of new object is *.app based on other object (0x1a3e1bd90) using this function.
A new object of type <class 'llama_index.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> at 0x11210f190 is calling an instrumented method <function BaseRetriever.retrieve at 0x1459cfce0>. The path of this call may be incorrect.
Guessing path of new object is *.app.retriever based on other object (0x1a3e1bd10) 

Groundedness scores: {'statement_0': 1.0, 'statement_1': 1.0, 'statement_2': 1.0}


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1a60fb850 is calling an instrumented method <function Refine.get_response at 0x1459ce340>. The path of this call may be incorrect.
Guessing path of new object is *.app._response_synthesizer based on other object (0x1a3e11a50) using this function.


Groundedness scores: {'statement_0': 0.5, 'statement_1': 0.3, 'statement_2': 0.3}


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1a64d8b50 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x1a3e693a0>. The path of this call may be incorrect.
Guessing path of new object is *.app based on other object (0x1a3e1bd90) using this function.


...

Groundedness scores: {'statement_0': 1.0}
Groundedness scores: {'statement_0': 1.0}


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1a60fb850 is calling an instrumented method <function Refine.get_response at 0x1459ce340>. The path of this call may be incorrect.
Guessing path of new object is *.app._response_synthesizer based on other object (0x1a3e11a50) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1a64d8b50 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x1a3e693a0>. The path of this call may be incorrect.
Guessing path of new object is *.app based on other object (0x1a3e1bd90) using this function.


Groundedness scores: {'statement_0': 1.0}
Groundedness scores: {'statement_0': 1.0}


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1a60fb850 is calling an instrumented method <function Refine.get_response at 0x1459ce340>. The path of this call may be incorrect.
Guessing path of new object is *.app._response_synthesizer based on other object (0x1a3e11a50) using this function.


Groundedness scores: {'statement_0': 1.0}
Groundedness scores: {'statement_0': 1.0}


## Add in more evals: Controversiality, Embedding Distance

In [13]:
# controversiality
f_controversiality = Feedback(openai.controversiality, name = "Controversiality").on_output() 

# embedding distance
from langchain.embeddings.openai import OpenAIEmbeddings
from trulens_eval.feedback import Embeddings

model_name = 'text-embedding-ada-002'

embed_model = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

embed = Embeddings(embed_model=embed_model)
f_embed_dist = (
    Feedback(embed.cosine_distance, name="Context Embedding Distance")
    .on_input()
    .on(TruLlama.select_source_nodes().node.text)
)

expanded_feedbacks = feedbacks + [f_controversiality, f_embed_dist]

tru_recorder_rag_sentencewindow = TruLlama(
        sentence_window_engine,
        app_id='RAG - Sentence Window',
        feedbacks=expanded_feedbacks
    )

✅ In controversiality, input text will be set to *.__record__.main_output or `Select.RecordOutput` .
✅ In cosine_distance, input query will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In cosine_distance, input document will be set to *.__record__.app.query.rets.source_nodes[:].node.text .


## Notice problem with high temperature

In [14]:
# lower temperature
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

sentence_index = build_sentence_window_index(
    document, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="sentence_index"
)

sentence_window_engine_lowtemp = get_sentence_window_query_engine(sentence_index)

tru_recorder_rag_sentencewindow_lowtemp = TruLlama(
        sentence_window_engine_lowtemp,
        app_id='RAG - Sentence Window - Low Temp',
        feedbacks=expanded_feedbacks
    )

In [15]:
# Run evaluation on 10 sample questions
with tru_recorder_rag_sentencewindow_lowtemp as recording:
    for question in eval_questions:
        response = sentence_window_engine_lowtemp.query(question)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1ac9eec10 is calling an instrumented method <function BaseQueryEngine.query at 0x138000a40>. The path of this call may be incorrect.
Guessing path of new object is *.app based on other object (0x1a3e1bd90) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1ac9eec10 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x1a3e693a0>. The path of this call may be incorrect.
Guessing path of new object is *.app based on other object (0x1a3e1bd90) using this function.
A new object of type <class 'llama_index.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> at 0x1a6fba190 is calling an instrumented method <function BaseRetriever.retrieve at 0x1459cfce0>. The path of this call may be incorrect.
Guessing path of new object is *.app.retriever based on other object (0x1a3e1bd10) 

Groundedness scores: {'statement_0': 1.0, 'statement_1': 1.0, 'statement_2': 1.0}
Groundedness scores: {'statement_0': 0.5, 'statement_1': 0.3, 'statement_2': 0.3}


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1a6fba290 is calling an instrumented method <function Refine.get_response at 0x1459ce340>. The path of this call may be incorrect.
Guessing path of new object is *.app._response_synthesizer based on other object (0x1a3e11a50) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1ac9eec10 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x1a3e693a0>. The path of this call may be incorrect.
Guessing path of new object is *.app based on other object (0x1a3e1bd90) using this function.


Groundedness scores: {'statement_0': 1.0}
Groundedness scores: {'statement_0': 1.0}


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1a6fba290 is calling an instrumented method <function Refine.get_response at 0x1459ce340>. The path of this call may be incorrect.
Guessing path of new object is *.app._response_synthesizer based on other object (0x1a3e11a50) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1ac9eec10 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x1a3e693a0>. The path of this call may be incorrect.
Guessing path of new object is *.app based on other object (0x1a3e1bd90) using this function.


...

Task queue full. Finishing existing tasks.


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1a6fba290 is calling an instrumented method <function Refine.get_response at 0x1459ce340>. The path of this call may be incorrect.
Guessing path of new object is *.app._response_synthesizer based on other object (0x1a3e11a50) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1ac9eec10 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x1a3e693a0>. The path of this call may be incorrect.
Guessing path of new object is *.app based on other object (0x1a3e1bd90) using this function.


Groundedness scores: {'statement_0': 1.0}
Groundedness scores: {'statement_0': 1.0}


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1a6fba290 is calling an instrumented method <function Refine.get_response at 0x1459ce340>. The path of this call may be incorrect.
Guessing path of new object is *.app._response_synthesizer based on other object (0x1a3e11a50) using this function.


Groundedness scores: {'statement_0': 1.0}
Groundedness scores: {'statement_0': 1.0}
